In [1]:
import tensorflow as tf
from tensorflow.keras import layers

# Vanilla LSTM

In [2]:
model = tf.keras.models.Sequential()

In [3]:
model.add(layers.LSTM(20, input_shape=(10, 1),return_sequences=True))

In [4]:
model.add(layers.TimeDistributed(layers.Dense(1, activation="sigmoid")))

In [44]:
model.compile(
    optimizer=tf.train.GradientDescentOptimizer(learning_rate=0.01),
    loss="binary_crossentropy",
    metrics=["acc"]
)

In [45]:
import numpy as np

In [46]:
n_timestamps = 10

In [47]:
X = np.random.rand(n_timestamps)

In [48]:
limit = n_timestamps / 4.0

In [49]:
y = (np.cumsum(X) > limit).astype(np.int32) 

In [50]:
y

array([0, 0, 0, 0, 0, 1, 1, 1, 1, 1], dtype=int32)

In [51]:
X = np.expand_dims(np.expand_dims(X, -1), 0)
y = np.expand_dims(np.expand_dims(y, -1), 0)

In [52]:
X.shape

(1, 10, 1)

In [53]:
y.shape

(1, 10, 1)

In [54]:
model.fit(X, y, epochs=10)

Epoch 1/10
1/1 [==============================] - 1s 793ms/step - loss: 0.6821 - acc: 0.5000
Epoch 2/10
1/1 [==============================] - 0s 14ms/step - loss: 0.6820 - acc: 0.5000
Epoch 3/10
1/1 [==============================] - 0s 16ms/step - loss: 0.6819 - acc: 0.5000
Epoch 4/10
1/1 [==============================] - 0s 15ms/step - loss: 0.6818 - acc: 0.5000
Epoch 5/10
1/1 [==============================] - 0s 16ms/step - loss: 0.6817 - acc: 0.5000
Epoch 6/10
1/1 [==============================] - 0s 17ms/step - loss: 0.6816 - acc: 0.5000
Epoch 7/10
1/1 [==============================] - 0s 17ms/step - loss: 0.6816 - acc: 0.5000
Epoch 8/10
1/1 [==============================] - 0s 17ms/step - loss: 0.6815 - acc: 0.5000
Epoch 9/10
1/1 [==============================] - 0s 19ms/step - loss: 0.6814 - acc: 0.5000
Epoch 10/10
1/1 [==============================] - 0s 19ms/step - loss: 0.6813 - acc: 0.5000


In [55]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 10, 20)            1760      
_________________________________________________________________
time_distributed (TimeDistri (None, 10, 1)             21        
Total params: 1,781
Trainable params: 1,781
Non-trainable params: 0
_________________________________________________________________


# BiLSTM

In [56]:
bilstm_model = tf.keras.models.Sequential()

In [57]:
bilstm_model.add(layers.Bidirectional(layers.LSTM(20, return_sequences=True), input_shape=(n_timestamps, 1)))
bilstm_model.add(layers.TimeDistributed(layers.Dense(1, activation="sigmoid")))

In [58]:
bilstm_model.compile(
    optimizer=tf.train.AdadeltaOptimizer(learning_rate=0.01),
    loss="binary_crossentropy",
    metrics=["acc"]
)

In [59]:
bilstm_model.fit(X, y, epochs=10)

Epoch 1/10
1/1 [==============================] - 2s 2s/step - loss: 0.6902 - acc: 0.5000
Epoch 2/10
1/1 [==============================] - 0s 14ms/step - loss: 0.6902 - acc: 0.5000
Epoch 3/10
1/1 [==============================] - 0s 17ms/step - loss: 0.6902 - acc: 0.5000
Epoch 4/10
1/1 [==============================] - 0s 16ms/step - loss: 0.6902 - acc: 0.5000
Epoch 5/10
1/1 [==============================] - 0s 16ms/step - loss: 0.6902 - acc: 0.5000
Epoch 6/10
1/1 [==============================] - 0s 19ms/step - loss: 0.6902 - acc: 0.5000
Epoch 7/10
1/1 [==============================] - 0s 18ms/step - loss: 0.6901 - acc: 0.5000
Epoch 8/10
1/1 [==============================] - 0s 18ms/step - loss: 0.6901 - acc: 0.5000
Epoch 9/10
1/1 [==============================] - 0s 22ms/step - loss: 0.6901 - acc: 0.5000
Epoch 10/10
1/1 [==============================] - 0s 20ms/step - loss: 0.6901 - acc: 0.5000


In [61]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 10, 20)            1760      
_________________________________________________________________
time_distributed (TimeDistri (None, 10, 1)             21        
Total params: 1,781
Trainable params: 1,781
Non-trainable params: 0
_________________________________________________________________


In [62]:
bilstm_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional (Bidirectional (None, 10, 40)            3520      
_________________________________________________________________
time_distributed_1 (TimeDist (None, 10, 1)             41        
Total params: 3,561
Trainable params: 3,561
Non-trainable params: 0
_________________________________________________________________


# BLSTM for frames

In [64]:
frame_dim = 1024
player_dim = 3472

In [63]:
num_hidden_states = 256
embedding_dim = 256

In [73]:
n_timestamps = 24

In [76]:
model = tf.keras.models.Sequential()
model.add(layers.Embedding(frame_dim, embedding_dim, input_length=n_timestamps))
model.add(layers.Bidirectional(layers.LSTM(num_hidden_states, return_sequences=True), input_shape=(n_timestamps, embedding_dim)))

In [77]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 24, 256)           262144    
_________________________________________________________________
bidirectional_5 (Bidirection (None, 24, 512)           1050624   
Total params: 1,312,768
Trainable params: 1,312,768
Non-trainable params: 0
_________________________________________________________________


# Classification Network

In [78]:
import tensorflow.keras.backend as K

In [308]:
batch_size=5

In [408]:
k = 11
num_players = 10

In [484]:
ft = layers.Input(shape=(n_timestamps, frame_dim))
pt = layers.Input(shape=(num_players, n_timestamps, player_dim))

ft_embed = layers.Dense(embedding_dim, activation="relu")(ft)
pt_embed = layers.Dense(embedding_dim, activation="relu")(pt)

h_f = layers.Bidirectional(layers.LSTM(num_hidden_states, return_sequences=True), input_shape=(n_timestamps, embedding_dim))(ft_embed)
h_f_repeated = layers.Lambda(lambda x: K.repeat_elements(K.expand_dims(x, 1), rep=num_players, axis=1))(h_f)

h_p = layers.TimeDistributed(layers.Masking(mask_value=0., input_shape=(n_timestamps, embedding_dim)))(pt_embed)
h_p = layers.TimeDistributed(layers.Bidirectional(layers.LSTM(num_hidden_states, return_sequences=True), input_shape=(n_timestamps, embedding_dim)))(pt_embed)

attention_probs = layers.Dense(embedding_dim * 2, activation="softmax")(layers.Concatenate()([h_f_repeated, h_p]))

merge = layers.Multiply()([h_p, attention_probs])
a_t = layers.Lambda(lambda x: K.sum(x, axis=1))(merge)

c_t = layers.Concatenate()([h_f, a_t])
h_e = layers.LSTM(num_hidden_states, return_sequences=True, input_shape=(n_timestamps, 2 * embedding_dim))(c_t)

outputs = layers.Dense(k, activation="linear")(h_e)


In [485]:
def squared_hinge_loss(y_true, y_pred):
    # y_true 100x11, y_pred 100x24x11
    # print(y_pred.get_shape())
    L = 0.5 * K.sum(K.square(K.maximum(0., 1. - y_true * y_pred)), axis=[-1, -2])
    return K.mean(L)

In [486]:
class_model = tf.keras.Model(inputs=[ft, pt], outputs=outputs)

In [487]:
class_model.compile(
    optimizer=tf.train.AdamOptimizer(learning_rate=0.01),
    loss=squared_hinge_loss
)

In [488]:
class_model.summary(line_length=150)

______________________________________________________________________________________________________________________________________________________
Layer (type)                                     Output Shape                     Param #           Connected to                                      
input_128 (InputLayer)                           (None, 24, 1024)                 0                                                                   
______________________________________________________________________________________________________________________________________________________
dense_158 (Dense)                                (None, 24, 256)                  262400            input_128[0][0]                                   
______________________________________________________________________________________________________________________________________________________
input_129 (InputLayer)                           (None, 10, 24, 3472)             0           

## Mock Data

In [478]:
N_SAMPLES = 100
N_CLASS = 11

In [448]:
frame = np.random.rand(N_SAMPLES, n_timestamps, frame_dim)
player = np.random.rand(N_SAMPLES, num_players, n_timestamps, player_dim)

In [449]:
labels = -1 * np.ones(shape=(N_SAMPLES, N_CLASS))
classes = np.random.randint(0, N_CLASS, N_SAMPLES)
labels[np.arange(len(labels)), classes] = 1

In [450]:
labels = labels.reshape(labels.shape[0], 1, labels.shape[1])
labels = labels.repeat(24, axis=1)

In [451]:
labels.shape

(100, 24, 11)

In [489]:
class_model.fit([frame, player], labels, epochs=5, batch_size=100)

Epoch 1/5
100/100 [==============================] - 38s 379ms/step - loss: 134.3285
Epoch 2/5
100/100 [==============================] - 8s 76ms/step - loss: 257.6184
Epoch 3/5
100/100 [==============================] - 6s 61ms/step - loss: 160.9807
Epoch 4/5
100/100 [==============================] - 7s 70ms/step - loss: 85.5073
Epoch 5/5
100/100 [==============================] - 7s 68ms/step - loss: 64.4176
